#  GEOCODER (ArcGIS), Folium and Choropleth Maps<a id="top"></a>
---

This is a short tutorial/quick start guide on how to obtain location data using **ArcGIS**, create interactive and Choropleth maps with **Folium**.<br>
  -  **ArcGIS** is a great utility in geocoder for obtaining the latitude and longitude  of a location.   
website:  https://www.esri.com/en-us/arcgis/about-arcgis/overview
<br><br>
  -  **Folium** is a Python library for creating interactive maps.   
website:  https://python-visualization.github.io/folium/


## Table Of Content
1. [ArcGIS](#arc)<br>
1.1  [Search with ArcGIS](#arc_search)<br>
1.2  [Define ArcGIS Function](#arc_func)<br>
1.3  [Execute ArcGIS Function](#arc_exec)<br>
1.4  [Invalid Location and ArcGIS](#arc_invalid)<br>
2. [Folium Maps](#fol)<br>
      2.1 [World Map](#fol_world)<br>
      2.2 [Markers](#fol_marker)<br>
3. [Choropleth Maps](#choro)<br>
3.1  [Create Dataframe](#choro_df)<br>
3.2  [Read geoJSON File](#choro_json)<br>
3.3  [Create Choropleth Map](#choro_plot)<br>


-  - -
 * **Lat/Lng Conversion**:  Folium works with **decimal** values for Latitude and Longitude.  Use the formula below if Latitude and Longitude are only available in degree/minutes/seconds:
    
    `Latitude  (dec) = degrees + (minutes/60) + (seconds/3600)`<br>
    `Longitude (dec) = degrees + (minutes/60) + (seconds/3600)`
<br>

 * **Install Folium**:  Jupyter supports folium.  For other applications/IDE (e.g. PyCharm, Spyder, Sublime, etc.), use "pip install folium --upgrade" or respective "install" variations.

<br>
I will keep adding more info/code as I go along.  Please upvote if you found this useful :-)

###   Import Libraries

In [ ]:
import pandas as pd
import folium          #  folium libraries
from   folium.plugins import MarkerCluster

try:
    import geocoder    #  geocoder
except:
    ! pip install geocoder
    import geocoder    #  geocoder


#  Kaggle directories
import os
print(os.listdir("../input"))
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

[top](#top)

---
#  1.  ArcGIS <a id="arc"></a>

###  1.1  Search with ArcGIS<a id="arc_search"></a>
Use **".latlng"** extension with **geocoder.arcgis** to get the latitude and longitude.

In [ ]:
geocoder.arcgis('dallas, texas').latlng    # get lat & lng coord

###  1.2  Define ArcGIS Function<a id="arc_func"></a>
Define function to get latitude and longitude.

In [ ]:
def arc_latlng(location):
    g = geocoder.arcgis('{}'.format(location))
    lat_lng_coords = g.latlng
    print(location,lat_lng_coords)
    return lat_lng_coords

arc_latlng('dallas, texas')     #  test arc_latlng

###  1.3  Execute ArcGIS Function<a id="arc_exec"></a>
Use the get_latlng function to get the latitude and longitude of several locations and save the information in a dataframe.  ARCGIS can take location names, zip/postal codes, landmark, etc.   
<br>
Sometimes, the dataframe columns do not maintain their order.  Verify that the column order is correct after dataframe is created.

In [ ]:
#  location list
#  10001 is the zip code of Manhattan, New York, US
#  M9B   is a postal code in Toronto, Canada
#  Everest is Mt. Everest in Nepal
location = ['10001','Tokyo','Sydney','Beijing','Karachi','Dehli', 'Everest','M9B','Eiffel Tower','Sao Paulo','Moscow']


#  call get_latlng function
loc_latlng = [arc_latlng(location) for location in location]


#  create dataframe for the results
df = pd.DataFrame(data = loc_latlng, columns = {'Latitude','Longitude'})
df.columns = ['Latitude','Longitude']  #  correct column order
df['Location'] = location              #  add location names

###  1.4  Invalid Location and ArcGIS<a id="arc_invalid"></a>
Invalid location will return a ***"None"*** value and will throw an error when creating the dataframe.

In [ ]:
invalid_loc = ['london','berlin','0902iuey7','999paris']  # 3 & 4 are invalid
invalid_latlng = [arc_latlng(invalid_loc) for invalid_loc in invalid_loc]

*Again, dataframe creation will fail because of the* "None".

[top](#top)

---
#  2.  Folium Maps <a id="fol"></a>

###  2.1  World Map<a id="fol_world"></a>
Plotting the locations from ArcGIS section.

In [ ]:
df

Note the following components in the Folium map:

  *  **location** - lat/lng can be any valid value
  *  **tiles** - style of map
  *  **zoom_start** - higher the number, closer the zoom
  *  **popup** - text when a marker is clicked

In [ ]:
#  center map on mean of Latitude/Longitude
map_world = folium.Map(location=[df.Latitude.mean(), df.Longitude.mean()], tiles = 'stamenterrain', zoom_start = 2)

#  add Locations to map
for lat, lng, label in zip(df.Latitude, df.Longitude, df.Location):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='Blue',
        fill_color='Yellow',
        fill_opacity=0.6
        ).add_to(map_world)

#  display interactive map
map_world

#  save map to local machine, open in any browser
#  map_world.save("C:\\ ... <path> ... \map_world_NYC.html")

###  2.1  Markers<a id="fol_mark"></a>
**CircleMarker** and **MarkerCluster** are two of the most common markers in Folium.  CircleMarker was used in the previous example.  Let's create a dataframe to use with MarkerCluster with locations around New York City.

In [ ]:
locationNYC = ['Empire State Building','Central Park','Wall Street','Brooklyn Bridge','Statue of Liberty','Rockefeller Center', 'Guggenheim Museum','Metlife Building','Times Square','United Nations Headquarters','Carnegie Hall']
locNYC_latlng = [arc_latlng(locationNYC) for locationNYC in locationNYC]

dfNY = pd.DataFrame(data = locNYC_latlng, columns = {'Latitude','Longitude'})
dfNY.columns = ['Latitude','Longitude']
dfNY['Location'] = locationNYC

Create Folium map with:
*  **CircleMarker** with world locations
*  **MarkerCluster** with New York landmarks 

Zoom in on New York City to see the cluster open up.  Click on the icon for landmark names.

In [ ]:
map_world_NYC = folium.Map(location=[df.Latitude.mean(), df.Longitude.mean()],
                       tiles = 'openstreetmap', 
                       zoom_start = 1)

#  CIRCLE MARKERS
#------------------------------
for lat, lng, label in zip(df.Latitude, df.Longitude, df.Location):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='black',
        fill_color='red',
        fill_opacity=0.6
        ).add_to(map_world_NYC)
#------------------------------

    
#  MARKERS CLUSTERS
#------------------------------
marker_cluster = MarkerCluster().add_to(map_world_NYC)
for lat, lng, label in zip(dfNY.Latitude, dfNY.Longitude, dfNY.Location):
    folium.Marker(location=[lat,lng],
            popup = label,
            icon = folium.Icon(color='green')
    ).add_to(marker_cluster)

map_world_NYC.add_child(marker_cluster)
#------------------------------

#  display map
map_world_NYC         

[top](#top)

---
# 3.  Choropleth Maps <a id="choro"></a>

**Choropleth maps** provide a visualization of statistical measurements, such as population density or per-capita income, overlaid on a geographic area.  A **geoJSON** file that defines the areas/boundaries of the state, county, or country is required.

###  3.1  Create Dataframe<a id="choro_df"></a>
`suicide rates from 1986 to 2016` dataset will be used to create a dataframe that contains 'country' and 'suicides/100k pop' for the year 2013.

  *  Some of country names will be updated to match the geoJSON file
  *  Once the data for 2013 has been created, the 'year' column can be ignored
  *  Data is available for only a limited number of countries

In [ ]:
dfs = pd.read_csv('../input/suicide-rates-overview-1985-to-2016/master.csv')  # suicide rates dataset

dfs = dfs[dfs['year'] == 2013]
dfs = dfs[['country','year','suicides/100k pop']].groupby('country').sum()
dfs.reset_index(inplace=True)

#  update names to match names in geoJSON file
dfs.replace({
        'United States':'United States of America',
        'Republic of Korea':'South Korea',
        'Russian Federation':'Russia'},
        inplace=True)

dfs.head()

###  3.2  Read geoJSON File<a id="choro_json"></a>
`world-countries` dataset will be used for the geoJSON file.

The geoJSON file contains the shape of the country in multiple latitude/logitude entries:
`{"type":"Feature","properties":{"name":"Austria"},"geometry":{"type":"Polygon","coordinates":[[[16.979667,48.123497],[16.903754,47.714866],...<deleted>`

NOTE:  Choropleth map will use the object `feature.properties.name` as a key.

In [ ]:
world_geo = os.path.join('../input/worldcountries', 'world-countries.json')

###  3.3  Create Choropleth Map<a id="choro_plot"></a>
Choropleth Map is created for *'suicides/100k pop'* per *'country'*.

Note the following components in the Choropleth Map:
  *  **geo_data** - geoJSON file
  *  **data** - dataframe for suicide rates
  *  **columns**  - dataframe columns 'country' and 'suicides/100k pop'
  *  **key_on**  - from geoJSON file: `feature.properties.name`

In [ ]:
world_choropelth = folium.Map(location=[0, 0], tiles='Cartodb Positron',zoom_start=2)

world_choropelth.choropleth(
    geo_data=world_geo,
    data=dfs,
    columns=['country','suicides/100k pop'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Suicide rates per 100k Population (2013)')

# display map
world_choropelth

###  EXTRA
Availble **Folium 'tiles':**<br>
tiles = 'cartodbdark_matter',<br>
'cartodbpositron',<br>
'cartodbpositronnolabels',<br>
'cartodbpositrononlylabels',<br>
'cloudmade',<br>
'mapbox',<br>
'mapboxbright',<br>
'mapboxcontrolroom',<br>
'openstreetmap',<br>
'stamenterrain',<br>
'stamentoner',<br>
'stamentonerbackground',<br>
'stamentonerlabels',<br>
'stamenwatercolor'.<br>
<br><br>
Availble **Folium 'fill_color':**<br>
fill_color = default 'blue'<br>
https://www.kaggle.com/asimislam/python-colors-color-cmap-palette

---
[top](#top)